In [1]:
import os
import json
import geojson
import urllib2
import pandas as pd
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
%matplotlib inline

In [2]:
os.listdir('.')

['.DS_Store',
 '.ipynb_checkpoints',
 'Archive',
 'cameras.json',
 'convert-php-to-geojson.ipynb',
 'new-data.php',
 'nyc.json']

In [3]:
with open('new-data.php') as f:
    data = json.load(f)

df = json_normalize(data['markers'])
df = df[['id', 'content', 'latitude', 'longitude']]
df.head()

,id,content,latitude,longitude
0,368,1 Ave @ 110 St,40.79142677512476,-73.93807411193848
1,360,1 Ave @ 124 St,40.800426144169315,-73.93155097961426
2,361,1 Ave @ 23 St,40.7359741672444,-73.97828578948975
3,550,1 Ave @ 42 St,40.74803725830298,-73.9694881439209
4,940,1 Ave @ 62 St,40.761501,-73.960542


In [4]:
def data2geojson(df, outfile):
    features = []
    df.apply(lambda x: features.append(
        geojson.Feature(geometry=geojson.Point((float(x['longitude']),
                                                float(x['latitude']))),
                        properties=dict(name=x['id'],
                                        cctv=x['cctv'],
                                        description=x['content']))),
            axis=1)
    with open(outfile, 'w+') as out:
        geojson.dump(geojson.FeatureCollection(features),
                     out, sort_keys=True)

In [5]:
def get_cctv_id(idx):
    url = 'http://dotsignals.org/google_popup.php?cid=' + str(idx)
    try: 
        page = urllib2.urlopen(url)
        soup = BeautifulSoup(page)
        return filter(lambda x: x.endswith('.jpg'), str(soup).split("'"))[0].split('/')[-1].rstrip('.jpg').lstrip('cctv')
    except:
        return 'No Response'

In [6]:
df['cctv'] = df['id'].apply(get_cctv_id)

In [7]:
data2geojson(df, 'cameras.json')